In [1]:
%load_ext autoreload

In [2]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

if not "../data_processing" in sys.path:
    sys.path.append("../data_processing")

%autoreload
import pca

## TEST #1: Run PCA with 80% of WTG02 as training and 20% as testing

In [8]:
path = '/Volumes/OsvikExtra/VibrationData/RMS_dataset/GbxHssRr_RMS_power>2500_WTG02.csv'
data = pd.read_csv(path)
data.head()

,AvgPower,ActPower,AvgRotSpeed,WindSpeed,NacelleDirection,GbxHssRr_RMS_0,GbxHssRr_RMS_1,GbxHssRr_RMS_2,GbxHssRr_RMS_3,GbxHssRr_RMS_4,...,GbxHssRr_RMS_40,GbxHssRr_RMS_41,GbxHssRr_RMS_42,GbxHssRr_RMS_43,GbxHssRr_RMS_44,GbxHssRr_RMS_45,GbxHssRr_RMS_46,GbxHssRr_RMS_47,GbxHssRr_RMS_48,GbxHssRr_RMS_49
0,2818.207520,3291.300049,1487.871218,14.2,226.699997,0.404392,0.258909,0.811581,0.344739,1.497359,...,1.101426,1.969857,1.778854,2.673142,3.386242,2.750154,1.419267,0.809299,0.444020,0.272188
1,2528.062988,2454.500000,1471.267571,10.6,231.699997,0.532580,0.244108,1.221325,0.346431,1.477624,...,1.106000,2.131452,1.824046,2.082840,2.086788,1.563331,0.788437,0.436628,0.294574,0.181487
2,2500.473145,1772.500000,1432.996560,8.4,284.600006,0.478452,0.210531,1.090423,0.360744,0.946055,...,0.967446,1.962500,1.463750,1.369159,1.247210,1.092017,0.519553,0.320830,0.221710,0.146560
3,2555.221924,2492.899902,1465.589029,9.0,132.000000,0.512141,0.226217,1.160659,0.358593,0.996903,...,1.068143,2.376538,1.979047,2.345064,2.218439,1.995184,0.850978,0.474515,0.348262,0.198423
4,3065.522217,3304.800049,1476.364735,12.7,245.000000,0.386640,0.276154,1.027501,0.356752,1.272245,...,1.155076,1.865711,1.766413,2.519427,3.441490,2.888474,1.341962,0.759380,0.538935,0.295528


In [9]:
data = data.drop(columns=['ActPower'])
def train_test_split(df, percentage):
    split_index = int(np.floor(df.shape[0]) * percentage)
    train = df[:split_index]
    test = df[split_index:].reset_index(drop=True)
    return train, test

train, test = train_test_split(data, 0.8)

In [10]:
print(train.shape)
print(test.shape)

(228, 54)
(58, 54)


## Scale with StandardScaler

In [11]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [12]:
test_pca_reconstructed, train_pca_reconstructed = pca.pca_calc(train_scaled, test_scaled, 0.95)

The number of components is: 18
The training reonstruction variance is: 0.046404543685996634
The testing reonstruction variance is: 0.09852230121728728
MAE loss on train data using pca: 0.16226539990476746
MAE loss on test data using pca: 0.2171445650893616


## TEST #2: Run PCA with WTG02 as training set and WTG03 as test set
Calculates mae in pca.py <br>
Ovservations: <br>
TEST #2 gives a higher training MAE Loss than TEST #1 but not as bad as TEST #4, #5 and #6

In [13]:
wtg02 = data
wtg03_path = '/Volumes/OsvikExtra/VibrationData/RMS_dataset/GbxHssRr_RMS_power>2500_WTG03.csv' 
wtg03 = pd.read_csv(wtg03_path)
wtg03 = wtg03.drop(columns=['ActPower'])

#Scale with StandardScaler
scaler = StandardScaler()
wtg02_scaled = scaler.fit_transform(wtg02)
wtg03_scaled = scaler.transform(wtg03)

#PCA
wtg02_pca_reconstructed, wtg03_pca_reconstructed = pca.pca_calc(wtg02_scaled, wtg03_scaled, 0.95)

The number of components is: 19
The training reonstruction variance is: 0.046779727255690434
The testing reonstruction variance is: 1.5749871200020158
MAE loss on train data using pca: 0.1612167038828315
MAE loss on test data using pca: 0.9383249140495296


## TEST #3: Run PCA with WTG02 and 50% of WTG03 as training and the remaining as testing
Calculates mae in pca.py <br>
Ovservations: <br>
TEST #3 gives a lower training MAE Loss than TEST #2 <br>
TEST #3 gives about the same MAE training Loss as TEST #1


In [14]:
wtg03_train, wtg03_test = train_test_split(wtg03, 0.5)
wtg02_train = wtg02.append(wtg03_train)

scaler = StandardScaler()
wtg02_train_scaled = scaler.fit_transform(wtg02_train)
wtg03_test_scaled = scaler.transform(wtg03_test)

wtg02_train_pca_reconstructed, wtg03_test_pca_reconstructed = pca.pca_calc(wtg02_train_scaled, 
                                                                           wtg03_test_scaled, 
                                                                           0.95)

The number of components is: 19
The training reonstruction variance is: 0.048583668826653575
The testing reonstruction variance is: 0.08874307258757708
MAE loss on train data using pca: 0.16388003203123022
MAE loss on test data using pca: 0.22631195819395855


## TEST #4: Run PCA with WTG01 as training and WTG02 as testing
Calculates mae in pca.py <br>
Ovservations: <br>
Training MAE Loss is incredibly bad


In [ ]:
wtg01_path = '/Volumes/OsvikExtra/VibrationData/RMS_dataset/GbxHssRr_RMS_power>2500_WTG01.csv' 
wtg01 = pd.read_csv(wtg01_path)
wtg01 = wtg01.drop(columns=['ActPower'])

In [ ]:
scaler = StandardScaler()
wtg01_scaled = scaler.fit_transform(wtg01)
wtg02_scaled = scaler.transform(wtg02)
wtg01_pca_reconstructed, wtg02_pca_reconstructed = pca.pca_calc(wtg01_scaled, 
                                                                wtg02_scaled, 
                                                                0.95)

## TEST #5: Run PCA with WTG01 as training and WTG04 as testing
Calculates mae in pca.py <br>
Ovservations: <br>
Training reconstruction is also incredibly bad - about the same as TEST #4

In [ ]:
wtg04_path = '/Volumes/OsvikExtra/VibrationData/RMS_dataset/GbxHssRr_RMS_power>2500_WTG04.csv' 
wtg04 = pd.read_csv(wtg04_path)
wtg04 = wtg04.drop(columns=['ActPower'])

In [ ]:
scaler = StandardScaler()
wtg01_scaled = scaler.fit_transform(wtg01)
wtg04_scaled = scaler.transform(wtg04)
wtg01_pca_reconstructed, wtg04_pca_reconstructed = pca.pca_calc(wtg01_scaled, 
                                                                wtg04_scaled, 
                                                                0.95)

## TEST #6: Run PCA with WTG01 as training and WTG03 as testing
Calculates mae in pca.py <br>
Ovservations: <br>
Training reconstruction is also incredibly bad - about the same as TEST #4 and TEST #5

In [ ]:
scaler = StandardScaler()
wtg01_scaled = scaler.fit_transform(wtg01)
wtg03_scaled = scaler.transform(wtg03)
wtg01_pca_reconstructed, wtg03_pca_reconstructed = pca.pca_calc(wtg01_scaled, 
                                                                wtg03_scaled, 
                                                                0.95)